# Load necessary moduls and settings

In [ ]:
'''poker engine'''

# pylint: disable=E1101, E1601, W0612

import gc
import json
import mysql.connector
from pypokerengine.api.emulator import Emulator

import funcs_db

gc.collect()

In [ ]:
json_data = open(file='./settings.json', mode='r')
settings = json.load(json_data)
json_data.close()

# Prepare poker database for new dataset

In [ ]:
poker_db = mysql.connector.connect(user=settings['sql_user'],\
                                   host=settings['sql_host'],\
                                   database=settings['sql_database'])
sql_path = settings['sql_path']
database = settings['sql_database']

In [ ]:
tables = funcs_db.sql_select_tables(poker_db=poker_db, database=database)
for table in tables:
    funcs_db.sql_delete_allrows(poker_db=poker_db, database=database, table=table)

# Prepare poker game for new plays

## Basic settings

In [ ]:
emulator = Emulator()

In [ ]:
player_num = settings['player_num']
max_round = settings['max_round']
small_blind_amount = settings['small_blind_amount']
ante_amount = settings['ante_amount']
stack = settings['stack']

players_info = {
    'uuid-0': {'name': 'player0', 'stack': stack},
    'uuid-1': {'name': 'player1', 'stack': stack},
    'uuid-2': {'name': 'player2', 'stack': stack},
    'uuid-3': {'name': 'player3', 'stack': stack},
    'uuid-4': {'name': 'player4', 'stack': stack},
    'uuid-5': {'name': 'player5', 'stack': stack}
}

summary = list()
index = funcs_db.sql_games_max_id(poker_db=poker_db, database=database)

## Prepare database

In [ ]:
emulator.set_game_rule(player_num=player_num,\
                       max_round=max_round,\
                       small_blind_amount=small_blind_amount,\
                       ante_amount=ante_amount)
initial_state = emulator.generate_initial_game_state(players_info)
for i, player in enumerate(initial_state['table'].seats.players):
    funcs_db.sql_insert_games(poker_db=poker_db,\
                              database=database,\
                              index=index + 1,\
                              player_num=player_num,\
                              small_blind_amount=small_blind_amount,\
                              ante_amount=ante_amount,\
                              uuid=player.uuid,\
                              name=player.name,\
                              stack=player.stack,\
                              position=i,\
                              card1='',\
                              card2='',\
                              hand_db_format='',\
                              flop1='',\
                              flop2='',\
                              flop3='',\
                              turn='',\
                              river='',\
                              final_stack=0)

## Update database

In [ ]:
game_state, events = emulator.start_new_round(initial_state)
for i, player in enumerate(game_state['table'].seats.players):
    card_list = [player.hole_card[0].RANK_MAP[player.hole_card[0].rank] +\
                 player.hole_card[0].SUIT_MAP[player.hole_card[0].suit],\
                 player.hole_card[1].RANK_MAP[player.hole_card[1].rank] +\
                 player.hole_card[1].SUIT_MAP[player.hole_card[1].suit]
    ]
    card_list.sort()
    card1 = card_list[0]
    card2 = card_list[1]
    if card1[-1] == card2[-1]:
        hand_db_format = card1[:-1] + card2[:-1] + 's'
    else:
        hand_db_format = card1[:-1] + card2[:-1] + 'o'

    funcs_db.sql_update_games_cards(poker_db=poker_db,\
                                    database=database,
                                    index=index + 1,\
                                    uuid=player.uuid,\
                                    card1=card1,\
                                    card2=card2,\
                                    hand_db_format=hand_db_format)

## Prepare smallblind and bigblind phase

In [ ]:
nr = settings['nr']
step = settings['step']
pot = settings['pot']
for i, player in enumerate(game_state['table'].seats.players):
    try:
        if player.serialize()[4][0]['action'] in ['SMALLBLIND', 'BIGBLIND']:
            if player.serialize()[4][0]['action'] == 'SMALLBLIND':
                amount = small_blind_amount
            else:
                amount = 2 * small_blind_amount
            
            funcs_db.sql_insert_history(poker_db=poker_db,\
                                        database=database,\
                                        phase='preflop',\
                                        nr=nr,\
                                        step=step,\
                                        uuid=player.serialize()[1],\
                                        position=i,\
                                        stack=settings['stack'],\
                                        stack_range=funcs_db.range_stack(stack=settings['stack'],\
                                                                         small_blind_amount=small_blind_amount),\
                                        pot=pot,\
                                        pot_range=funcs_db.range_pot(pot=pot,\
                                                                     small_blind_amount=small_blind_amount),\
                                        flop1='',\
                                        flop2='',\
                                        flop3='',\
                                        turn='',\
                                        river='',\
                                        action=player.serialize()[4][0]['action'],\
                                        amount=amount,\
                                        amount_potrate=funcs_db.range_potrate(amount=amount,\
                                                                              pot=pot,\
                                                                              action=player.serialize()[4][0]['action']),\
                                        new_stack=settings['stack'] - amount,\
                                        new_stack_range=funcs_db.range_stack(stack=settings['stack'] - amount,\
                                                                             small_blind_amount=small_blind_amount),\
                                        new_pot=pot + amount,\
                                        new_pot_range=funcs_db.range_pot(pot=pot + amount,\
                                                                         small_blind_amount=small_blind_amount))

            nr = nr + 1
            step = step + 1
            pot = pot + amount
    except:
        pass

# Game phase

## Hint
{database}.possible_moves.bet_amount: implies the "until size" of some action, e.g. "raise til 100" (already debt amount + raise amount)  
{database}.history.amount: net change in pot caused by some action

In [ ]:
while events[-1]['type'] != 'event_game_finish':
    
    # data source
    data = events[-1]
    
    # calculate parameters from current game state
    phase = data['round_state']['street']
    position = data['round_state']['next_player']
    uuid = data['uuid']
    for item in data['round_state']['seats']:
        if item['uuid'] == data['uuid']:
            stack = item['stack']

    side = settings['side']
    if data['round_state']['pot']['side'] != []:
        for amount in data['round_state']['pot']['side']:        
            side = side + amount['amount']

    pot = data['round_state']['pot']['main']['amount'] + side
    valid_actions = funcs_db.valid_actions_check(actions=data['valid_actions'],\
                                                 phase=phase,\
                                                 position=position,\
                                                 stack=stack,\
                                                 small_blind_amount=small_blind_amount)
    community_cards = funcs_db.community_cards_eval(board=data['round_state']['community_card'])

    # calculate action and bet_amount and apply them on game state
    action, bet_amount, decision =\
        funcs_db.decision_point_based_action(poker_db=poker_db,\
                                             database=database,\
                                             phase=phase,\
                                             nr=nr,\
                                             step=step,\
                                             position=position,\
                                             stack=stack,\
                                             pot=pot,\
                                             flop1=community_cards[0],\
                                             flop2=community_cards[1],\
                                             flop3=community_cards[2],\
                                             turn=community_cards[3],\
                                             river=community_cards[4],\
                                             valid_actions=valid_actions)
    
    game_state, events = emulator.apply_action(game_state=game_state,\
                                               action=action,\
                                               bet_amount=bet_amount)
    summary.append(decision)
    
    # calculate new parameters from current game state
    if events[-1]['type'] == 'event_game_finish':
        data = events[0]
    else:
        data = events[-1]
    
    new_phase = data['round_state']['street']    
    new_side = settings['side']
    if data['round_state']['pot']['side'] != []:
        for amount in data['round_state']['pot']['side']:        
            new_side = new_side + amount['amount']

    new_pot = data['round_state']['pot']['main']['amount'] + new_side
    for item in data['round_state']['seats']:
        if item['uuid'] == uuid and data['round_state']['street'] != 'showdown':
            new_stack = item['stack']
        elif item['uuid'] == uuid and data['round_state']['street'] == 'showdown':
            new_stack = stack - new_pot + pot
        else:
            pass
    
    amount = stack - new_stack
    
    # insert action in history table with calculated old and new parameters
    funcs_db.sql_insert_history(poker_db=poker_db,\
                                database=database,\
                                phase=phase,\
                                nr=nr,\
                                step=step,\
                                uuid=uuid,\
                                position=position,\
                                stack=stack,\
                                stack_range=funcs_db.range_stack(stack=stack,\
                                                                 small_blind_amount=small_blind_amount),\
                                pot=pot,\
                                pot_range=funcs_db.range_pot(pot=pot,\
                                                             small_blind_amount=small_blind_amount),\
                                flop1=community_cards[0],\
                                flop2=community_cards[1],\
                                flop3=community_cards[2],\
                                turn=community_cards[3],\
                                river=community_cards[4],\
                                action=action.upper(),\
                                amount=amount,\
                                amount_potrate=funcs_db.range_potrate(amount=amount,\
                                                                      pot=pot,\
                                                                      action=action.upper()),\
                                new_stack=new_stack,\
                                new_stack_range=funcs_db.range_stack(stack=new_stack,\
                                                                     small_blind_amount=small_blind_amount),\
                                new_pot=new_pot,\
                                new_pot_range=funcs_db.range_pot(pot=new_pot,\
                                                                 small_blind_amount=small_blind_amount))

    # increase nr parameter - nr has to be set to 0 when phase changes
    if phase == new_phase:
        nr = nr + 1
    else:
        nr = 0
    
    step = step + 1

In [ ]:
funcs_db.summarize(poker_db=poker_db, database=databse, final_stacks=events[-1]['players'], summary=summary)

In [ ]:
# when player's stack is zero then rebuy, mulitply small blind with random integer in range min-max starting stack
# prepare script to run it for n loop - finsih stack is starting stack in new round